In [42]:
#import packages
import json
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [43]:
#loading JSON file into dictionary
filepath = "student_vectors_n_task_10_n_limit_10000.json"
student_vectors = json.load(open(filepath))

for j in student_vectors['V0D6D2D5O7']:
    if(j['qual_id'] == '1zsCldT4p8.set2.JiTh5Wk0bD'):
        print(j)

{'ccssm': 'CCSS.Math.3.NF.A.1', 'untouched': False, 'qual_id': '1zsCldT4p8.set2.JiTh5Wk0bD', 'time_spent': 31732, 'correct': False, 'ts': 1473782027923.0, 'task_id': '1zsCldT4p8.set2', 'second_try': False}
{'ccssm': 'CCSS.Math.3.NF.A.1', 'untouched': False, 'qual_id': '1zsCldT4p8.set2.JiTh5Wk0bD', 'time_spent': 41516, 'correct': True, 'ts': 1473782037722.0, 'task_id': '1zsCldT4p8.set2', 'second_try': True}


In [44]:
#collect all qual_ids, ccssm_labels, task_ids in separate lists
all_qual_ids = []
all_ccssm_labels = []
all_task_ids = []
for i in student_vectors:
    for j in student_vectors[i]:
        all_qual_ids.append(j['qual_id'])
        all_ccssm_labels.append(j['ccssm'])
        all_task_ids.append(j['task_id'])

#make a set of unique values from the above lists
unique_ids = set(all_qual_ids)
unique_labels = set(all_ccssm_labels)
unique_tasks = set(all_task_ids)
print("Number of unique labels in this dataset " + str(len(unique_labels))) #unique labels
print("Number of unique lessons/tasks in this dataset "+str(len(unique_tasks))) #unique lessons
print("Number of unique questions in this dataset "+str(len(unique_ids))) #this is the length of bit vector (number of unique qual_ids)

#generate vectors to give to fit_transform in multilabelbinarizer to further generate unique 1-hot encoding
transform_ids = []
for i in unique_ids:
    transform_ids.append([i])

transform_labels = []
for i in unique_labels:
    transform_labels.append([i])

Number of unique labels in this dataset 4
Number of unique lessons/tasks in this dataset 10
Number of unique questions in this dataset 612


In [45]:
#generate dictionary that maps labels and qual_ids to their respective 1-hot encoding
enc = MultiLabelBinarizer()
qual_ids_1hot = enc.fit_transform(transform_ids)
qual_ids_classes = enc.classes_
qual_ids_dict = dict(zip(unique_ids, qual_ids_1hot))
labels_1hot = enc.fit_transform(transform_labels)
labels_classes = enc.classes_
labels_dict = dict(zip(unique_labels,labels_1hot))

In [53]:
#generate final input vector to feed the network
final_input = []
second_try_flag = False
for i in student_vectors: #loop over all the students
    interactions_vector = []
    for j in student_vectors[i]: #loop over all the interactions of student 'i'
        #assuming there is no qual_id that has 'untouched': True
        qual_id_vector = qual_ids_dict[j['qual_id']]
        label_vector = labels_dict[j['ccssm']]
        combined_vector = np.concatenate([qual_id_vector,label_vector])
        #scaling the inputs
        if(j['correct'] == True and j['second_try'] == False):
            #the student did correctly in the first try itself!
            combined_vector *= 3
            interactions_vector.append(combined_vector)
        elif(j['correct'] == False and j['second_try'] == False):
            #student will be given second try, nothing will be appended to interactions vector
            second_try_flag = True
        elif(j['correct'] == True and j['second_try'] == True and second_try_flag == True):
            #student does correctly in the second try
            second_try_flag = False
            combined_vector *= 2
            interactions_vector.append(combined_vector)
        elif(j['correct'] == False and j['second_try'] == True and second_try_flag == True):
            #student did incorrectly the second try
            second_try_flag = False
            interactions_vector.append(combined_vector)
    final_input.append(interactions_vector)
print(final_input[0])

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0

In [ ]:
#final output vectors, still to decide what it would be
final_output = []